In [1]:
import numpy as np
import matplotlib.pyplot as plt
from parse import parse

In [176]:
train_images, train_labels = parse('facedata/facedatatrain', 'facedata/facedatatrainlabels', False)
validation_images, validation_labels = parse('facedata/facedatavalidation', 'facedata/facedatavalidationlabels', False)
test_images, test_labels = parse('facedata/facedatatest', 'facedata/facedatatestlabels', False)

In [81]:
train_images, train_labels = parse('digitdata/trainingimages', 'digitdata/traininglabels', True)
validation_images, validation_labels = parse('digitdata/validationimages', 'digitdata/validationlabels', True)
test_images, test_labels = parse('digitdata/testimages', 'digitdata/testlabels', True)

In [235]:
base_rate_true = np.bincount(train_labels)/train_labels.shape[0]
print(base_rate_true)

[0.51884701 0.48115299]


In [ ]:
num_black_pixels = np.sum(images, axis = (1,2))

In [277]:
def black_pixels_in_zone(images, labels, roi):
    images = images[:, roi[0]:roi[2], roi[1]:roi[3]]

    pixels_in_zone = images[0].shape[0]*images[0].shape[1]
    # print(pixels_in_zone)
    black_pixels_prob = np.zeros((pixels_in_zone , 2))

    for index in range(0, pixels_in_zone):
        black_pixels_prob[index, 1] = np.sum(np.sum(images[labels.astype(bool)], axis = (1, 2)) == index)/labels[labels.astype(bool)].shape[0]
        black_pixels_prob[index, 0] = np.sum(np.sum(images[np.invert(labels.astype(bool))], axis = (1, 2)) == index)/images[np.invert(labels.astype(bool))].shape[0]

    return black_pixels_prob

In [236]:
arr= np.array([0,1,2,3,4,5])

In [240]:
arr[1:None]

array([1, 2, 3, 4, 5])

In [304]:
def zone_sum_prob(images, labels, num_of_zones):
    div = np.sqrt(num_of_zones)

    height = int(images[0].shape[0] // div)
    width = int(images[0].shape[1] // div)
    # number_of_zones = height * width
    # print(type(height))
    probs = []

    i = 0 
    j = 0 - width
    for index in range(0, num_of_zones):

        if index % div == 0:
            j += width
            i = 0
        else:
            i += height

        zone_prob = black_pixels_in_zone(images, labels, [i, j, i + height, j + width])
        probs.append(zone_prob)

        
        
    # print(len(probs))
    for index, prob in enumerate(probs):
        # print(prob.shape)
        if prob.shape[0] == 0:
            print(index)
    # print(probs)
    return np.stack(probs, axis = 0)




In [305]:
def zone_sum(images, labels, num_of_zones):
    div = np.sqrt(num_of_zones)

    height = int(images[0].shape[0] // div)
    width = int(images[0].shape[1] // div)
    # number_of_zones = height * width
    # print(type(height))
    sums = []

    i = 0 
    j = 0 - width
    for index in range(0, num_of_zones):
        if index % div == 0:
            j += width
            i = 0
        else:
            i += height

        zone_sum  = np.sum(images[:,i:i+height, j:j+width], axis =(1, 2))
        sums.append(zone_sum)

    return np.stack(sums, axis = 0)

In [306]:
zone_probs = zone_sum_prob(train_images, train_labels, 100)

In [307]:
zone_probs.shape

(100, 42, 2)

In [308]:
zone_sums = zone_sum(train_images, train_labels, 100)

In [313]:
zone_sums.dtype

dtype('int32')

In [314]:
zone_sums.choose(zone_probs)

ValueError: Need at least 0 and at most 32 array objects.

In [ ]:
zone_probs()

In [220]:
num_pixels_prob_1 = black_pixels_in_zone(train_images, train_labels, [0, 0, 7, 6])

In [227]:
num_pixels_prob_1[np.argmax(num_pixels_prob_1[:,0])]

array([0.48717949, 0.53917051])

In [229]:
predictions = num_pixels_prob_1[num_pixels, 0] * base_rate_true[0] < num_pixels_prob_1[num_pixels, 1] * base_rate_true[0]
np.sum(predictions == train_labels)/train_labels.shape[0]

0.5188470066518847

In [209]:
base_rate_true

array([0.51884701, 0.48115299])

In [187]:
num_pixels = np.sum(train_images, axis = (1, 2))

In [188]:
num_pixels.shape

(451,)

In [231]:
pixels_in_image = train_images[0].shape[0]*train_images[0].shape[1]
num_pixels_prob = np.zeros((pixels_in_image, 2))
for index in range(0, pixels_in_image):
    num_pixels_prob[index, 1] = np.sum(np.sum(train_images[train_labels.astype(bool)], axis = (1, 2)) == index)/train_labels[train_labels.astype(bool)].shape[0]
    num_pixels_prob[index, 0] = np.sum(np.sum(train_images[np.invert(train_labels.astype(bool))], axis = (1, 2)) == index)/train_images[np.invert(train_labels.astype(bool))].shape[0]

In [232]:
predictions = num_pixels_prob[num_pixels, 0] * base_rate_true[0] < num_pixels_prob[num_pixels, 1] * base_rate_true[0]
np.sum(predictions == train_labels)/train_labels.shape[0]

0.8248337028824834

In [210]:
num_pixels_validation = np.sum(validation_images, axis = (1, 2))

In [233]:
predictions = num_pixels_prob[num_pixels_validation, 0] * base_rate_true[0] < num_pixels_prob[num_pixels_validation, 1] * base_rate_true[0]
np.sum(predictions == validation_labels)/validation_labels.shape[0]

0.6046511627906976

In [203]:
train_labels[train_labels.astype(bool)].shape[0]

217

In [199]:
train_images[train_labels].shape[0]

451

In [204]:
np.sum(np.sum(train_images[train_labels], axis = (1, 2)) == 728)/train_labels[train_labels.astype(bool)].shape[0]

1.0

In [182]:
np.sum(np.sum(train_images[train_labels], axis = (1, 2)) == 422)/train_images[train_labels].shape[0]

0.0

In [186]:
np.argmax(num_pixels_prob[:, 1])

0